<a href="https://colab.research.google.com/github/sahupra1357/LLMAgents/blob/main/Groq_LangroidAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q --upgrade langroid &> /dev/null
!pip show langroid

Name: langroid
Version: 0.49.1
Summary: Harness LLMs with Multi-Agent Programming
Home-page: 
Author: 
Author-email: Prasad Chalasani <pchalasani@gmail.com>
License: MIT
Location: /usr/local/lib/python3.11/dist-packages
Requires: adb-cloud-connector, aiohttp, async-generator, bs4, cerebras-cloud-sdk, colorlog, docstring-parser, duckduckgo-search, exa-py, faker, fakeredis, fire, gitpython, google-api-python-client, google-genai, groq, grpcio, halo, jinja2, json-repair, lxml, nest-asyncio, nltk, onnxruntime, openai, pandas, prettytable, pydantic, pygithub, pygments, pymupdf4llm, pyparsing, pytest-rerunfailures, python-dotenv, python-magic, pyyaml, qdrant-client, rank-bm25, redis, requests, requests-oauthlib, rich, thefuzz, tiktoken, trafilatura, typer, wget
Required-by: 


In [ ]:
import os
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

In [ ]:
import langroid as lr
import langroid.language_models as lm
from langroid.agent.tools.duckduckgo_search_tool import DuckduckgoSearchTool
from langroid.utils.configuration import settings
settings.notebook = True

MODEL = "groq/llama3-70b-8192"

In [ ]:
llm_config = lm.OpenAIGPTConfig(
      chat_model=MODEL, # or lr.OpenAIChatModel.GPT4_TURBO
      chat_context_length=8192,
      temperature=0,
      max_output_tokens=200,
      timeout=45,
  )

In [ ]:
assistant_config = lr.ChatAgentConfig(
    system_message="""
    You are a resourceful assistant, able to think step by step to answer
    complex questions from the user. You must break down complex questions into
    simpler questions that can be answered by a web search. You must ask me
    (the user) each question ONE BY ONE, and I will do a web search and send you
    a brief answer. Once you have enough information to answer my original
    (complex) question, you MUST say DONE and present the answer to me.
    """,
    llm=llm_config,
    vecdb=None,
)
assistant_agent = lr.ChatAgent(assistant_config)

assistant_task = lr.Task(
    assistant_agent,
    name="Assistant",
    llm_delegate=True,
    single_round=False,
    interactive=False,
)


                        using fake redis client
2025-03-29 16:19:03 - WARNING - REDIS_PASSWORD, REDIS_HOST, REDIS_PORT not set in .env file,
                        using fake redis client


In [ ]:
search_tool_handler_method = DuckduckgoSearchTool.default_value("request")

search_agent_config = lr.ChatAgentConfig(
    llm=llm_config,
    vecdb=None,
    system_message=f"""
    You are a web-searcher. For any question you get, you must use the
    `{search_tool_handler_method}` tool/function-call to get up to 5 results.
    I WILL SEND YOU THE RESULTS; DO NOT MAKE UP THE RESULTS!!
    Once you receive the results, you must compose a CONCISE answer
    based on the search results and say DONE and show the answer to me,
    in this format:
    DONE [... your CONCISE answer here ...]
    IMPORTANT: YOU MUST WAIT FOR ME TO SEND YOU THE
    SEARCH RESULTS BEFORE saying you're DONE.
    """,
)
search_agent = lr.ChatAgent(search_agent_config)
search_agent.enable_message(DuckduckgoSearchTool)


search_task = lr.Task(
    search_agent,
    name="Searcher",
    llm_delegate=True,
    single_round=False,
    interactive=False,
)

In [ ]:
assistant_task.add_sub_task(search_task)
question = "Was Liszt born before Wagner?"
assistant_task.run(question)

>>> Starting Agent Assistant (1) llama3-70b-8192 

To answer this question, I need to know the birth year of each composer. Let's start with Liszt. 

What is the birth year of Franz Liszt?

Stats: N_MSG=2, TOKENS: in=109, out=34, max=200, ctx=8192, COST: now=$0.0000, cumul=$0.0000, tot=$0.0000 
(llama3-70b-8192)

...|>>> Starting Agent Searcher (1) llama3-70b-8192 

...|

...

...|>>> Starting Agent Searcher (5) llama3-70b-8192 

...|

TOOL: 
{
    "request": "duckduckgo_search",
    "query": "What is the birth year of Richard Wagner?",
    "num_results": 5
}

...|Stats: N_MSG=6, TOKENS: in=1104, out=38, max=200, ctx=8192, COST: now=$0.0000, cumul=$0.0000, tot=$0.0000 
(llama3-70b-8192)

RatelimitException: https://links.duckduckgo.com/d.js?q=What+is+the+birth+year+of+Richard+Wagner%3F&kl=wt-wt&l=wt-wt&p=&s=0&df=&vqd=4-157228487782857929890682563830021885009&bing_market=wt-WT&ex=-1 202 Ratelimit